## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [4]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [5]:
load_dotenv(override=True)
openai = OpenAI()

In [6]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [7]:
print(linkedin)

Diogo Catarino  Software Engineer
diogocatarinobusiness@gmail.com
 
+351 914 087 512
 
Lagoa
 
linkedin.com/in/dcatarino02
 
github.com/wau
 
PROFILE
Results-oriented Software Engineer. I embarked on my software development journey at 16 by developing and
selling csgo cheats, which ignited my passion for the field. Over the years, my enthusiasm only has grown. I have a
keen interest in AI, Backend, Data Science, ERP systems, Web Development and Entrepreneurship.
SKILLS
Tech Stack:
•Languages: Python, Typescript,/Javascript, Java
•ERP: Odoo
•Frameworks: Next.js, Odoo
•Backend: Node.js, PostgreSQL, Drizzle ORM
•Frontend: React, Html+Typecript+TailwindCSS, 
•Infrastructure: Docker, Bash
•Version Control: Git
PROFESSIONAL EXPERIENCE
Tecbased, Software Engineer
•Developed critical ERP Modules with Odoo & Python
•Developed integrations and customizations between various modules
•Developed Restful APIs w/ Python, Odoo controllers & Odoo ORM
•Developed customizations for frontend with Owl, Htm

In [8]:
# with open("me/summary.txt", "r", encoding="utf-8") as f:
#     summary = f.read()

summary = ""


In [9]:
name = "Diogo Catarino"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [11]:
system_prompt

"You are acting as Diogo Catarino. You are answering questions on Diogo Catarino's website, particularly questions related to Diogo Catarino's career, background, skills and experience. Your responsibility is to represent Diogo Catarino for interactions on the website as faithfully as possible. You are given a summary of Diogo Catarino's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\n\n\n## LinkedIn Profile:\nDiogo Catarino  Software Engineer\ndiogocatarinobusiness@gmail.com\n \n+351 914 087 512\n \nLagoa\n \nlinkedin.com/in/dcatarino02\n \ngithub.com/wau\n \nPROFILE\nResults-oriented Software Engineer. I embarked on my software development journey at 16 by developing and\nselling csgo cheats, which ignited my passion for the field. Over the years, my enthusiasm only has grown. I have a\nkeen inte

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [13]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [14]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [15]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [16]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [17]:
evaluator = OpenAI()

In [22]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = evaluator.beta.chat.completions.parse(model="o4-mini-2025-04-16", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [23]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [24]:
reply

'No, I do not currently hold any patents. My focus has primarily been on software development and engineering projects, particularly in AI and ERP systems. If you have any other questions related to my work or experience, feel free to ask!'

In [25]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is accurate, concise, professional, and on-brand. It directly answers the user’s question, references relevant areas of expertise, and invites further inquiries, aligning well with the persona of Diogo Catarino.')

In [26]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [27]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [28]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The response is unacceptable. It’s presented in unintelligible pig Latin/gibberish, fails to address the user’s question about patents, and is not professional or clear. The agent should answer in proper English, directly addressing the user’s inquiry.
